In [41]:
import pandas as pd
import numpy as np
import openpyxl

from datetime import datetime
from pathlib import Path
import os, re, locale, json

pd.set_option('display.float_format', '{:,.2f}'.format)

In [42]:
def start():
    options = {
        'display': {
            'float_format': '{:,.2f}'.format,
            'max_columns': None,
            'max_colwidth': 25,
            'expand_frame_repr': False,  # Don't wrap to multiple pages
            'max_rows': 14,
            'max_seq_items': 50,         # Max length of printed sequence
            'precision': 4,
            'show_dimensions': True
        },
        'mode': {
            'chained_assignment': None   # Controls SettingWithCopyWarning
        }
    }
    for category, option in options.items():
        for op, value in option.items():
            pd.set_option(f'{category}.{op}', value)  # Python 3.6+
start()

In [68]:
df = pd.read_excel('data/2021-02-13_GuiaPetroleo.xlsx', sheet_name=1, usecols=('A:O'))

C:\Users\Mauricio\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:308: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [69]:
df.shape

(2502, 15)

In [70]:
df.dropna(subset=['CNPJ'], inplace=True)

In [71]:
df.loc[df['Razão Social'].str.contains('JX')]

,Categoria,Segmento,Razão Social,Fantasia,CNPJ,Endereço,Cep,País,Estado,Cidade,Telefone 1,Telefone 2,Email 1,Email 2,Site
171,E&P,Cias de Petróleo,JX Nippon Oil & Gás E...,JX Nippon (Brasil),19500000000000,Rua Projetada 2000,20220-297,Brasil,RJ,Rio de Janeiro,NaN,NaN,NaN,NaN,http://www.nex.jx-gro...


In [76]:
df['CNPJ'] = df['CNPJ'].astype(str)

In [78]:
type(df.loc[171]['CNPJ'])

str

In [79]:
branches = df[df['CNPJ'].str.contains('^(?!.*/0001).*$', regex=True)]

In [80]:
branches[branches['CNPJ'] == np.nan]

,Categoria,Segmento,Razão Social,Fantasia,CNPJ,Endereço,Cep,País,Estado,Cidade,Telefone 1,Telefone 2,Email 1,Email 2,Site


In [9]:
branches.drop_duplicates(subset=['CNPJ'], keep='first')

,Categoria,Segmento,Razão Social,Fantasia,CNPJ,Endereço,Cep,País,Estado,Cidade,Telefone 1,Telefone 2,Email 1,Email 2,Site
122,Produtos,Aquecedores,Aker Solutions do Bra...,Aker Solutions,05.876.349/0002-96,Av. Rio Branco 01,20090-003,Brasil,RJ,Rio de Janeiro,(21) 2122-4401,NaN,sales@akersolutions.com,info@akersolutions.com,https://akersolutions...
148,Serviços,Revestimento Anticorr...,Akzo Nobel Ltda.,Akzo Nobel Ltda.,60.561.719/0059-40,"Av. Paiva Neves, 999",24426-140,Brasil,RJ,São Gonçalo,(21) 2628-3992,(11) 2173-7152,adrianna.vigorito@akz...,NaN,https://www.akzonobel...
149,Serviços,Meio Ambiente,AlBriggs Defesa Ambie...,AlBriggs,04.050.400/0028-82,"Av. Jabaquara, 3060",04046-500,Brasil,SP,São Paulo,(11) 4059-9999,0800 772 2220,contato@albriggs.com.br,NaN,https://www.albriggs....
162,Organizações,Câmaras de Comércio,Câmara de Comércio Am...,Amcham Brasil,62.044.151/0018-47,"Praça Pio X, 15 - 5º ...",20040-020,Brasil,RJ,Rio de Janeiro,(21) 3213-9200,NaN,amchamrio@amchamrio.com,rio@amchambrasil.com.br,https://www.amcham.co...
171,Industrial,Refino,Amec Foster Wheeler B...,Amec Foster Wheeler,01.505.625/0002-59,Estr. São José do Mut...,27973-030,Brasil,RJ,Macaé,(22) 3518 0100,NaN,info@woodplc.com,help@woodplc.com,https://www.woodplc.com/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,Recursos Humanos,MBA,Universidade Iguaçu,Unig,30.834.196/0007-76,Av. Abílio Augusto Tá...,26210-000,Brasil,RJ,Nova Iguaçu,0800 021-2013,(21) 2765-4000,contato@unig.br,NaN,http://www.unig.br/
2228,Produtos,Filtros,Veolia Water Technolo...,Veolia Water Solution...,96.591.128/0002-27,"Av Carlos Marcondes, ...",12241-421,Brasil,SP,São José Dos Campos,(11) 3888-8800,(11) 4617-4388,NaN,NaN,https://www.veoliawat...
2263,Produtos,Aço,Villares Metals S.A,Villares Metals,42.566.752/0004-07,R. Alfredo Dumont Vil...,13178-902,Brasil,SP,Sumaré,0800 190577,0800 707-0577,metals@villaresmetals...,NaN,http://www.villaresme...
2274,Produtos,Químicos,Weg Equipamentos Elet...,WEG Tintas,07.175.725/0010-50,Av. Prefeito Waldemar...,89256-900,Brasil,SC,Jaraguá do Sul,NaN,(47) 3276-4000,info-br@weg.net,NaN,http://www.weg.net/in...


In [10]:
branches.index

Int64Index([ 122,  123,  124,  125,  126,  127,  128,  129,  130,  131,
            ...
            2204, 2228, 2229, 2230, 2231, 2232, 2263, 2274, 2275, 2321],
           dtype='int64', length=209)

In [11]:
headquarters = df.drop(index=branches.index)

In [12]:
headquarters

,Categoria,Segmento,Razão Social,Fantasia,CNPJ,Endereço,Cep,País,Estado,Cidade,Telefone 1,Telefone 2,Email 1,Email 2,Site
0,Produtos,Compressores,A Geradora Aluguel de...,A Geradora,33.845.322/0001-90,Av. Antonio Carlos Ma...,41800-700,Brasil,BA,Salvador,0800 333 5000,NaN,ageradora@ageradora.c...,contato@ageradora.com.br,https://www.ageradora...
1,Produtos,Geradores,A Geradora Aluguel de...,A Geradora,33.845.322/0001-90,Av. Antonio Carlos Ma...,41800-700,Brasil,BA,Salvador,0800 333 5000,NaN,ageradora@ageradora.c...,contato@ageradora.com.br,https://www.ageradora...
2,Produtos,Máquinas,A Geradora Aluguel de...,A Geradora,33.845.322/0001-90,Av. Antonio Carlos Ma...,41800-700,Brasil,BA,Salvador,0800 333 5000,NaN,ageradora@ageradora.c...,contato@ageradora.com.br,https://www.ageradora...
3,Serviços,Locação de Equipamentos,A Geradora Aluguel de...,A Geradora,33.845.322/0001-90,Av. Antonio Carlos Ma...,41800-700,Brasil,BA,Salvador,0800 333 5000,NaN,ageradora@ageradora.c...,contato@ageradora.com.br,https://www.ageradora...
4,Serviços,Naval,Aalborg Comércio e Se...,Aalborg Marine Supply,30.153.894/0001-10,R. Sinézio Trindade C...,27920-490,Brasil,RJ,Macaé,(22) 2773-5005,NaN,aalborg@aalborg.com.br,NaN,http://www.aalborg.co...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2423,Serviços,Bancos,Scania Banco S.A.,Scania Banco,11.417.016/0001-10,"Av. José Odorizzi, 151",09810-000,Brasil,SP,São Bernardo do Campo,(11) 4344-9333,-,-,-,www.scaniabanco.com.br
2424,Serviços,Bancos,Scotiabank Brasil S.A...,Scotiabank Brasil,29.030.467/0001-66,"Av. Brig. Faria Lima,...",01452-000,Brasil,SP,São Paulo,(11) 2202-8100,(11) 4052-8100,contactus@scotiabank.com,callcenter@scotiabank...,www.br.scotiabank.com
2425,Serviços,Bancos,Travelex Banco de Câm...,Travelex Bank,04.913.129/0001-41,Av. Engenheiro Luiz C...,04571-010,Brasil,SP,São Paulo,0800 771 1878,-,ouvidoria@confidencec...,-,https://www.travelexb...
2426,Serviços,Bancos,UBS Brasil Banco de I...,UBS Brasil Banco de I...,18.520.834/0001-93,Av.Brigadeiro Faria L...,04538-132,Brasil,SP,São Paulo,(11) 2767-6500,-,contact@ubs.com,help@ubs.com,www.ubs.com


In [13]:
headquarters['CNPJ'].value_counts()

09.202.985/0001-02    43
07.583.589/0001-47    35
55.445.191/0001-97    29
28.415.370/0001-09    20
01.113.887/0001-97    20
                      ..
48.795.256/0001-69     1
61.576.369/0001-31     1
40.368.151/0001-11     1
07.679.404/0001-00     1
03.215.790/0001-10     1
Name: CNPJ, Length: 782, dtype: int64

In [21]:
headquarters.drop_duplicates(subset=['CNPJ'], keep='first', inplace=True)

# Localizando a diferença em headquarters

In [16]:
hq_new = pd.read_excel('datasets/petroleo_empresas.xlsx', sheet_name='headquarters')

In [17]:
hq_new.shape

(783, 16)

In [25]:
hq_new['Unnamed: 0']

0         0
1         4
2         5
3        12
4        13
       ... 
778    2423
779    2424
780    2425
781    2426
782    2427
Name: Unnamed: 0, Length: 783, dtype: int64

In [34]:
idx_hq_new = hq_new['Unnamed: 0'].values.tolist()

In [35]:
idx_hq_new

[0,
 4,
 5,
 12,
 13,
 14,
 15,
 16,
 17,
 19,
 20,
 21,
 22,
 26,
 27,
 28,
 29,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 43,
 44,
 45,
 60,
 64,
 65,
 66,
 68,
 69,
 74,
 76,
 78,
 80,
 82,
 84,
 88,
 89,
 90,
 96,
 97,
 98,
 105,
 106,
 107,
 108,
 110,
 113,
 117,
 150,
 151,
 153,
 154,
 155,
 156,
 157,
 160,
 163,
 173,
 174,
 177,
 179,
 180,
 181,
 182,
 183,
 184,
 186,
 188,
 199,
 200,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 215,
 219,
 224,
 226,
 235,
 237,
 239,
 240,
 242,
 244,
 245,
 246,
 247,
 248,
 250,
 251,
 258,
 260,
 284,
 285,
 288,
 292,
 294,
 295,
 296,
 301,
 310,
 319,
 321,
 322,
 324,
 326,
 330,
 331,
 335,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 357,
 359,
 364,
 365,
 367,
 368,
 369,
 370,
 375,
 376,
 377,
 378,
 384,
 386,
 389,
 390,
 393,
 398,
 401,
 404,
 405,
 412,
 413,
 414,
 419,
 421,
 424,
 426,
 427,
 431,
 433,
 435,
 436,
 437,
 443,
 444,
 446,
 450,
 466,
 468,
 473,
 

In [36]:
headquarters.index.to_list()

[0,
 4,
 5,
 12,
 13,
 14,
 15,
 16,
 17,
 19,
 20,
 21,
 22,
 26,
 27,
 28,
 29,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 43,
 44,
 45,
 60,
 64,
 65,
 66,
 68,
 69,
 74,
 76,
 78,
 80,
 82,
 84,
 88,
 89,
 90,
 96,
 97,
 98,
 105,
 106,
 107,
 108,
 110,
 113,
 117,
 150,
 151,
 153,
 154,
 155,
 156,
 157,
 160,
 163,
 173,
 174,
 177,
 179,
 180,
 181,
 182,
 184,
 186,
 188,
 199,
 200,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 215,
 219,
 224,
 226,
 235,
 237,
 239,
 240,
 242,
 244,
 245,
 246,
 247,
 248,
 250,
 251,
 258,
 260,
 284,
 285,
 288,
 292,
 294,
 295,
 296,
 301,
 310,
 319,
 321,
 322,
 324,
 326,
 330,
 331,
 335,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 357,
 359,
 364,
 365,
 367,
 368,
 369,
 370,
 375,
 376,
 377,
 378,
 384,
 386,
 389,
 390,
 393,
 398,
 401,
 404,
 405,
 412,
 413,
 414,
 419,
 421,
 424,
 426,
 427,
 431,
 433,
 435,
 436,
 437,
 443,
 444,
 446,
 450,
 466,
 468,
 473,
 482,
 

In [37]:
for i in idx_hq_new:
    if i not in headquarters.index.to_list():
        print(i)

183
